In [1]:
import rospy as rp
from jupyros import ros3d
import jupyros as jr
from std_msgs.msg import String
from nav_msgs.msg import OccupancyGrid

In [2]:
rp.init_node('diagnotics_node')

### Check connection to ROS

In [3]:
rp.get_published_topics()

[['/map_metadata', 'nav_msgs/MapMetaData'],
 ['/move_base/global_costmap/costmap_updates', 'map_msgs/OccupancyGridUpdate'],
 ['/sonar', 'sensor_msgs/Range'],
 ['/move_base/current_goal', 'geometry_msgs/PoseStamped'],
 ['/camera/rgb/image_rect_color/theora/parameter_descriptions',
  'dynamic_reconfigure/ConfigDescription'],
 ['/move_base/parameter_descriptions',
  'dynamic_reconfigure/ConfigDescription'],
 ['/move_base/local_costmap/inflation_layer/parameter_descriptions',
  'dynamic_reconfigure/ConfigDescription'],
 ['/move_base/feedback', 'move_base_msgs/MoveBaseActionFeedback'],
 ['/move_base/result', 'move_base_msgs/MoveBaseActionResult'],
 ['/odom_gazebo', 'nav_msgs/Odometry'],
 ['/camera/rgb/image_rect_color/theora', 'theora_image_transport/Packet'],
 ['/tf', 'tf2_msgs/TFMessage'],
 ['/connected_clients', 'rosbridge_msgs/ConnectedClients'],
 ['/move_base/local_costmap/costmap', 'nav_msgs/OccupancyGrid'],
 ['/move_base/global_costmap/static_layer/parameter_updates',
  'dynamic_reco

### Checki if using simulation time

In [4]:
rp.get_param('/use_sim_time')
# rp.get_param_names()

True

In [5]:
# from rosgraph_msgs.msg import Clock
# jr.subscribe("/clock", Clock, lambda msg: print(msg))

### Check Transform from odom to base_footprint

In [6]:
import tf
listener = tf.TransformListener()

In [7]:
listener.getFrameStrings()

/opt/ros/noetic/lib/python3/dist-packages/tf/listener.py:97: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(self._buffer.all_frames_as_yaml()) or {}


['base_link',
 'base_footprint',
 'camera_depth_frame',
 'camera_link',
 'camera_depth_optical_frame',
 'body_link',
 'front_sonar',
 'laser',
 'odom',
 'wheel_front_left',
 'wheel_front_right',
 'wheel_left',
 'wheel_rear_left',
 'wheel_rear_right',
 'wheel_right']

In [8]:
listener.getLatestCommonTime("/base_footprint", "/base_footprint")

rospy.Time[148911000000]

In [9]:
listener.getLatestCommonTime("/odom", "/odom")

rospy.Time[149701000000]

In [10]:
listener.getLatestCommonTime("/map", "/odom")

rospy.Time[150001000000]

In [11]:
listener.lookupTransform('/base_footprint', '/base_link', rp.Time.now())

([0.0, 0.0, 0.0762], [0.0, 0.0, 0.0, 1.0])

In [12]:
listener.lookupTransform('/odom', '/base_footprint', rp.Time.now() - rp.Time(0.3))

([0.0010894230197967475, 0.004174279002459482, -1.9833095197763217e-05],
 [-1.3801416260900248e-05,
  2.032658779088859e-06,
  -0.0008401052072163548,
  0.9999996470142527])

In [13]:
listener.lookupTransform('/map', '/odom', rp.Time.now() - rp.Time(0.3))

([0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0])

In [14]:
listener.lookupTransform('/map', '/base_footprint', rp.Time.now() - rp.Time(0.3))

([0.0010992309003698007, 0.004212084576005477, -1.9833095201632342e-05],
 [-1.3801402300263185e-05,
  2.0327534615882503e-06,
  -0.000846965535544767,
  0.9999996412273208])

In [15]:
# from nav_msgs.msg import Odometry
# jr.subscribe("/odom", Odometry, lambda msg: print(msg))

### Show map

In [16]:
# jr.subscribe("/map", OccupancyGrid, lambda msg: print(msg))

In [17]:
from jupyros import ros3d

v = ros3d.Viewer()
v.objects = [ros3d.GridModel()]
v

Viewer(objects=[GridModel()])

In [18]:
rc = ros3d.ROSConnection()
tf_client = ros3d.TFClient(ros=rc, fixed_frame='/map')
occupancyMap = ros3d.OccupancyGrid(ros=rc, tf_client=tf_client, topic='/map', continuous=True)
localCostMap = ros3d.OccupancyGrid(ros=rc, tf_client=tf_client, topic='/move_base/local_costmap/costmap', continuous=True, color='#aaF000FF')
urdf = ros3d.URDFModel(ros=rc, tf_client=tf_client)
laser = ros3d.LaserScan(topic="/scan", ros=rc, tf_client=tf_client)
v.objects = [ros3d.GridModel(), occupancyMap, urdf, laser, localCostMap]